In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("processed.csv")
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [3]:
df.dropna(inplace=True)

In [4]:
import dagshub
dagshub.init(repo_owner='rajatchauhan99', repo_name='Youtube-Comment-Analysis-Chrome-Plugin', mlflow=True)

Accessing as rajatchauhan99

Initialized MLflow to track repo "rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin"

Repository rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin initialized!

In [5]:
# Set or create an experiment
import mlflow
mlflow.set_experiment("Exp 5 - Trying Different ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/96c047f2f48c4d168382939725e408d8', creation_time=1729342817379, experiment_id='5', last_update_time=1729342817379, lifecycle_stage='active', name='Exp 5 - Trying Different ML Algos with HP Tuning', tags={}>

In [6]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\rajat.chauhan\conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Random Forest

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'],
                                        random_state=42)

    # Log the best model with MLflow, passing the algo_name as "RandomForest"
    log_mlflow("RandomForest", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()


[I 2024-10-19 18:42:11,458] A new study created in memory with name: no-name-460093de-31bc-44dc-b27f-5a3acde6cdec
[I 2024-10-19 18:42:32,620] Trial 0 finished with value: 0.6920212765957446 and parameters: {'n_estimators': 142, 'max_depth': 19, 'min_samples_split': 9, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.6920212765957446.
[I 2024-10-19 18:42:57,224] Trial 1 finished with value: 0.6940425531914893 and parameters: {'n_estimators': 146, 'max_depth': 19, 'min_samples_split': 10, 'min_samples_leaf': 19}. Best is trial 1 with value: 0.6940425531914893.
[I 2024-10-19 18:43:11,873] Trial 2 finished with value: 0.7053191489361702 and parameters: {'n_estimators': 62, 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 1}. Best is trial 2 with value: 0.7053191489361702.
[I 2024-10-19 18:43:22,616] Trial 3 finished with value: 0.6458510638297872 and parameters: {'n_estimators': 126, 'max_depth': 7, 'min_samples_split': 14, 'min_samples_leaf': 6}. Best is trial 2 with va